# Using Keras to classify German Traffic Sign database

## Objectives
- Use the same architecture as in 2.6 but using keras
- Reuse the Data Input and Processing

In [1]:
import tensorflow as tf
import keras

import numpy as np
import cv2
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
tf.__version__

'1.0.0'

In [3]:
keras.__version__

'1.2.2'

<img align="left" src="files/old.png"> **Load data**

In [4]:
training_file = './data/train2.p'
testing_file = './data/test2.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

<img align="left" src="files/old.png"> **Standardize naming** 

In [5]:
xtr = train['features']
ytr = train['labels']

xva, xte, yva, yte = train_test_split(test['features'], test['labels'], test_size=0.5)

<img align="left" src="files/old.png"> **One hot encoding**

In [6]:
encoder = LabelBinarizer()
encoder.fit(ytr)
ytr = encoder.transform(ytr).astype(float)
yte = encoder.transform(yte).astype(float)
yva = encoder.transform(yva).astype(float)

<img align="left" src="files/old.png"> **Normalize the input**

In [7]:
def normalize(x_in,y_in):
    
    x_out = []
    y_out = []
    for i, (image, label) in enumerate(zip(x_in, y_in)):
        zeros = np.zeros((32,32,3))
        norm_image = cv2.normalize(image, zeros, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        x_out.append(norm_image)
        y_out.append(label)

    return np.array(x_out), np.array(y_out)

In [8]:
xtr, ytr = normalize(xtr,ytr)
xte, yte = normalize(xte,yte)
xva, yva = normalize(xva,yva)

<img align="left" src="files/new.png"> **Import keras libraries**

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

<img align="left" src="files/new.png"> **Define keras model**

In [10]:
model = Sequential()

model.add(Convolution2D(32, 5, 5, activation='relu', border_mode='same', input_shape=(32,32,3), dim_ordering='tf'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(64, 5, 5, activation='relu', border_mode='same', input_shape=(16,16,32), dim_ordering='tf'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

<img align="left" src="files/new.png"> **Train**

In [11]:
model.fit(xtr, ytr, batch_size=64, nb_epoch=20, verbose=2)

Epoch 1/20
12s - loss: 0.7298 - acc: 0.7927
Epoch 2/20
10s - loss: 0.1028 - acc: 0.9701
Epoch 3/20
10s - loss: 0.0537 - acc: 0.9858
Epoch 4/20
10s - loss: 0.0338 - acc: 0.9905
Epoch 5/20
10s - loss: 0.0266 - acc: 0.9918
Epoch 6/20
10s - loss: 0.0182 - acc: 0.9946
Epoch 7/20
10s - loss: 0.0231 - acc: 0.9932
Epoch 8/20
10s - loss: 0.0188 - acc: 0.9945
Epoch 9/20
10s - loss: 0.0149 - acc: 0.9960
Epoch 10/20
10s - loss: 0.0192 - acc: 0.9943
Epoch 11/20
10s - loss: 0.0106 - acc: 0.9972
Epoch 12/20
10s - loss: 0.0103 - acc: 0.9968
Epoch 13/20
10s - loss: 0.0152 - acc: 0.9961
Epoch 14/20
10s - loss: 0.0158 - acc: 0.9953
Epoch 15/20
10s - loss: 0.0069 - acc: 0.9979
Epoch 16/20
10s - loss: 0.0089 - acc: 0.9975
Epoch 17/20
10s - loss: 0.0060 - acc: 0.9980
Epoch 18/20
10s - loss: 0.0145 - acc: 0.9965
Epoch 19/20
10s - loss: 0.0098 - acc: 0.9973
Epoch 20/20
10s - loss: 0.0103 - acc: 0.9973


In [12]:
print model.evaluate(xte, yte, verbose=0)

[0.37875744427160468, 0.95312747431480971]


In [13]:
model.metrics_names

['loss', 'acc']